# Top-Level Consistency Checks

This notebook checks the top-level `tiles-SPECPROD.(fits|csv)` and `exposures-SPECPROD.(fits|csv)` files for self-consistency in prepartion for loading these files into a database.

## Imports

In [2]:
import os
import glob
from pytz import utc
import numpy as np
from astropy.io import fits
from astropy.table import Table
from astropy.time import Time
from desiutil.iers import freeze_iers
from desiutil.log import get_logger, DEBUG, INFO
from desispec.io.meta import faflavor2program
from desispec.io.util import checkgzip
import desispec.database.redshift as dsr
freeze_iers()

INFO:iers.py:82:freeze_iers: Freezing IERS table used by astropy time, coordinates.


## Important Setup

In [3]:
specprod = os.environ['SPECPROD'] = 'fuji'
overwrite = True

## Examine ztile files.

In [15]:
ztile_files = glob.glob(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, 'zcatalog', 'ztile-*.fits'))
for f in ztile_files:
    if 'cumulative' in f:
        key = 'LASTNIGHT'
    elif 'perexp' in f:
        key = 'EXPID'
    elif 'pernight' in f:
        key = 'NIGHT'
    else:
        key = 'SPGRPVAL'
    t = Table.read(f, format='fits', hdu='ZCATALOG')
    assert (t[key] == t['SPGRPVAL']).all()

In [7]:
ztile_files[8]

'/global/cfs/cdirs/desi/spectro/redux/fuji/zcatalog/ztile-sv1-dark-cumulative.fits'

In [11]:
cumulative = Table.read(ztile_files[8], format='fits', hdu='ZCATALOG')
assert (cumulative['LASTNIGHT'] == cumulative['SPGRPVAL']).all()

In [14]:
perexp = Table.read(ztile_files[6], format='fits', hdu='ZCATALOG')
assert (perexp['EXPID'] == perexp['SPGRPVAL']).all()

TARGETID,NIGHT,EXPID,SPGRPVAL,Z,ZERR,ZWARN,CHI2,COEFF [10],NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2,PETAL_LOC,DEVICE_LOC,LOCATION,FIBER,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,REF_EPOCH,LAMBDA_REF,FA_TARGET,FA_TYPE,OBJTYPE,FIBERASSIGN_X,FIBERASSIGN_Y,PRIORITY,SUBPRIORITY,OBSCONDITIONS,RELEASE,BRICKNAME,BRICKID,BRICK_OBJID,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,MASKBITS,SERSIC,SHAPE_R,SHAPE_E1,SHAPE_E2,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_RP_MEAN_MAG,PARALLAX,PHOTSYS,PRIORITY_INIT,NUMOBS_INIT,CMX_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,PLATE_RA,PLATE_DEC,TILEID,COADD_NUMEXP,COADD_EXPTIME,COADD_NUMNIGHT,COADD_NUMTILE,MEAN_DELTA_X,RMS_DELTA_X,MEAN_DELTA_Y,RMS_DELTA_Y,MEAN_FIBER_RA,STD_FIBER_RA,MEAN_FIBER_DEC,STD_FIBER_DEC,MEAN_PSF_TO_FIBER_SPECFLUX,MEAN_FIBER_X,MEAN_FIBER_Y,TSNR2_GPBDARK_B,TSNR2_ELG_B,TSNR2_GPBBRIGHT_B,TSNR2_LYA_B,TSNR2_BGS_B,TSNR2_GPBBACKUP_B,TSNR2_QSO_B,TSNR2_LRG_B,TSNR2_GPBDARK_R,TSNR2_ELG_R,TSNR2_GPBBRIGHT_R,TSNR2_LYA_R,TSNR2_BGS_R,TSNR2_GPBBACKUP_R,TSNR2_QSO_R,TSNR2_LRG_R,TSNR2_GPBDARK_Z,TSNR2_ELG_Z,TSNR2_GPBBRIGHT_Z,TSNR2_LYA_Z,TSNR2_BGS_Z,TSNR2_GPBBACKUP_Z,TSNR2_QSO_Z,TSNR2_LRG_Z,TSNR2_GPBDARK,TSNR2_ELG,TSNR2_GPBBRIGHT,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBBACKUP,TSNR2_QSO,TSNR2_LRG,ZCAT_NSPEC,ZCAT_PRIMARY
int64,int32,int32,int32,float64,float64,int64,float64,float64,int64,bytes6,bytes20,int64,float64,int16,int32,int64,int32,int32,float64,float64,float32,float32,float32,float32,int64,uint8,bytes3,float32,float32,int32,float64,int32,int16,bytes8,int32,int32,bytes4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,int64,bytes2,float32,float32,float32,float32,bytes1,int64,int64,int64,int64,int64,int64,float64,float64,int32,int16,float32,int16,int16,float32,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,bool
616089235992085010,20201216,68265,68265,0.9805607821235571,0.0001466618221216555,5,7387.7574267834425,-12.919637734019435 .. -0.9929909407343875,7821,GALAXY,--,10,1.4637096673250198,0,311,311,0,0,23.64704835712534,30.237699207256,0.0,0.0,0.0,5400.0,4294967296,4,SKY,81.98283,-286.4917,-1,0.8849026730392795,63,9010,0235p302,497013,530,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-0.0565887,-0.124679565,-0.18450871,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,--,0.0,0.0,0.0,0.0,--,-1,-1,4294967296,4294967296,0,0,23.64704835712534,30.237699207256,80615,1,900.0,1,1,-0.007,0.007,-0.006,0.006,23.647081267417626,0.0,30.237675802383013,0.0,0.7924254,81.974,-286.481,126.533356,0.075946644,24.026545,64.35186,410.45438,174.28282,2.0275366,0.70895755,9685.332,19.67003,1749.791,0.041074183,2010.0316,11144.427,6.2561483,28.98696,1.4749437e-05,65.14,2.8074712e-06,0.0,2875.9753,2.0592614e-05,13.42022,29.783068,9811.865,84.88598,1773.8175,64.39294,5296.4614,11318.71,21.703905,59.478985,4,False
39628483692856624,20201216,68265,68265,0.8841011806172813,6.520683633088104e-05,0,10462.785076379776,107.91414463257848 .. -20.72010564816314,7815,GALAXY,--,10,134.15092033147812,0,272,272,1,0,23.76534865647498,30.29102253628784,0.0,0.0,2020.9597,5400.0,1024,1,TGT,56.339264,-272.51822,3200,0.7279310459082589,1,9010,--,497014,1328,PSF,0.0556925,0.16067116,1.5628034,6.9378138,18.046345,7.403878,2415.7075,579.5788,91.570786,-1.0,-1.0,0.124920554,1.2150671,5.394095,0.12514971,1.2153398,5.3946676,0,0.0,0.0,0.0,0.0,0,--,0.0,0.0,0.0,0.0,S,3200,1,1024,0,0,0,23.76534865647498,30.29102253628784,80615,1,900.0,1,1,-0.007,0.007,-0.007,0.007,23.765381645047803,

## Test loading targeting data

In [3]:
os.environ['DESI_LOGLEVEL'] = 'DEBUG'
dsr.log = get_logger()
postgresql = dsr.setup_db(schema=specprod+'_target', overwrite=overwrite, hostname='nerscdb03.nersc.gov', verbose=True)

INFO:redshift.py:1178:setup_db: Begin creating tables.
2022-03-10 15:35:46,691 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-03-10 15:35:46,692 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-10 15:35:46,694 INFO sqlalchemy.engine.Engine select current_schema()
2022-03-10 15:35:46,694 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-10 15:35:46,696 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-03-10 15:35:46,696 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-10 15:35:46,698 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-10 15:35:46,700 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2022-03-10 15:35:46,701 INFO sqlalchemy.engine.Engine [generated in 0.00121s] {'schema': 'fuji_target', 'name': 'tractor'}
2022-03-10 15:35:46,703 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespa

In [4]:
loader = [{'filepaths': [os.path.join('/global/cscratch1/sd/ioannis/photocatalog', os.environ['SPECPROD'], 'targetphot-{specprod}.fits'.format(specprod=os.environ['SPECPROD'])),
                         os.path.join('/global/cscratch1/sd/ioannis/photocatalog', os.environ['SPECPROD'], 'targetphot-potential-targets-{specprod}.fits'.format(specprod=os.environ['SPECPROD'])),
                         os.path.join('/global/cscratch1/sd/ioannis/photocatalog', os.environ['SPECPROD'], 'targetphot-missing-{specprod}.fits'.format(specprod=os.environ['SPECPROD']))],
           'tcls': dsr.Target,
           'hdu': 'TARGETPHOT',
           'expand': {'DCHISQ': ('dchisq_psf', 'dchisq_rex', 'dchisq_dev', 'dchisq_exp', 'dchisq_ser',)},
           'q3c': 'ra',
           'chunksize': 100000,
           'maxrows': 0
           },]


In [ ]:
dsr.load_file(**(loader[0]))

INFO:redshift.py:780:load_file: Identified 3 files for ingestion.
INFO:redshift.py:785:load_file: Read 1979269 rows of data from /global/cscratch1/sd/ioannis/photocatalog/fuji/targetphot-fuji.fits HDU TARGETPHOT.
INFO:redshift.py:832:load_file: Integrity check complete on target.
INFO:redshift.py:837:load_file: Row filter applied on target.
INFO:redshift.py:846:load_file: Initial column conversion complete on target.
DEBUG:redshift.py:864:load_file: Expanding column 0 of DCHISQ (at index 9) to dchisq_psf.
DEBUG:redshift.py:864:load_file: Expanding column 1 of DCHISQ (at index 9) to dchisq_rex.
DEBUG:redshift.py:864:load_file: Expanding column 2 of DCHISQ (at index 9) to dchisq_dev.
DEBUG:redshift.py:864:load_file: Expanding column 3 of DCHISQ (at index 9) to dchisq_exp.
DEBUG:redshift.py:864:load_file: Expanding column 4 of DCHISQ (at index 9) to dchisq_ser.
DEBUG:redshift.py:867:load_file: ['release', 'brickid', 'brickname', 'brick_objid', 'morphtype', 'ra', 'ra_ivar', 'dec', 'dec_iva

## Consistency checks on exposures, frames and tiles

### Load two versions of tiles file

In [ ]:
tiles_fits = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'tiles-{specprod}.fits'), hdu='TILE_COMPLETENESS')

In [ ]:
tiles_fits

In [ ]:
tiles_csv = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'tiles-{specprod}.csv'), format='ascii.csv')

In [ ]:
tiles_csv

### Are the two tiles files self-consistent?

In [ ]:
for row in range(len(tiles_fits)):
    for col in tiles_fits.colnames:
        try:
            assert tiles_fits[row][col] == tiles_csv[row][col]
        except AssertionError:
            print(tiles_fits[row]['TILEID'], col, tiles_fits[row][col], tiles_csv[row][col])

### Load two versions of exposures file

In [ ]:
exposures_fits = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'), hdu='EXPOSURES')
frames = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'), hdu='FRAMES')

In [ ]:
exposures_fits

In [ ]:
exposures_csv = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.csv'), format='ascii.csv')

In [ ]:
exposures_csv

### Are the two exposures files self-consistent?

In [ ]:
for row in range(len(exposures_fits)):
    for col in exposures_fits.colnames:
        try:
            assert exposures_fits[row][col] == exposures_csv[row][col]
        except AssertionError:
            try:
                assert np.around(exposures_fits[row][col].astype(float), 1) == exposures_csv[row][col]
            except AssertionError:
                try:
                    assert np.around(exposures_fits[row][col].astype(float), 2) == exposures_csv[row][col]
                except AssertionError:
                    try:
                        assert np.around(exposures_fits[row][col].astype(float), 3) == exposures_csv[row][col]
                    except AssertionError:
                        print(exposures_fits[row]['TILEID'], col, exposures_fits[row][col], exposures_csv[row][col])

### What programs are present?

In [ ]:
np.unique(faflavor2program(exposures_fits['FAFLAVOR']))

In [ ]:
np.unique(exposures_fits['PROGRAM'])

In [ ]:
np.unique(exposures_fits['GOALTYPE'])

In [ ]:
np.unique(faflavor2program(tiles_fits['FAFLAVOR']))

In [ ]:
np.unique(tiles_fits['PROGRAM'])

In [ ]:
assert (faflavor2program(tiles_fits['FAFLAVOR']) == tiles_fits['PROGRAM']).all

In [ ]:
program = faflavor2program(exposures_fits['FAFLAVOR'])
assert (exposures_fits['PROGRAM'] == program).all()

In [ ]:
for survey in np.unique(exposures_fits['SURVEY']):
    print(f"'{survey}': ", np.unique(program[exposures_fits['SURVEY'] == survey]).tolist(), ',', sep='')

### Compare frames to exposures

In [ ]:
for expid in frames['EXPID']:
    assert expid in exposures_fits['EXPID']

In [ ]:
for k, expid in enumerate(exposures_fits['EXPID']):
    assert (frames['NIGHT'][frames['EXPID'] == expid] == exposures_fits[k]['NIGHT']).all()
    assert (frames['TILEID'][frames['EXPID'] == expid] == exposures_fits[k]['TILEID']).all()
    assert (frames['TILERA'][frames['EXPID'] == expid] == exposures_fits[k]['TILERA']).all()
    assert (frames['TILEDEC'][frames['EXPID'] == expid] == exposures_fits[k]['TILEDEC']).all()
    assert (frames['AIRMASS'][frames['EXPID'] == expid] == exposures_fits[k]['AIRMASS']).all()
    assert (frames['SEEING_ETC'][frames['EXPID'] == expid] == exposures_fits[k]['SEEING_ETC']).all()
    try:
        assert (frames['EFFTIME_ETC'][frames['EXPID'] == expid] == exposures_fits[k]['EFFTIME_ETC']).all()
    except AssertionError:
        print('EFFTIME_ETC', expid, exposures_fits[k]['TILEID'], exposures_fits[k]['EFFTIME_ETC'], frames[column][frames['EFFTIME_ETC'] == expid].tolist())
    # assert (frames['TSNR2_GPBDARK'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBDARK']).all()
    # assert (frames['TSNR2_ELG'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_ELG']).all()
    # assert (frames['TSNR2_GPBBRIGHT'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBBRIGHT']).all()
    # assert (frames['TSNR2_LYA'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_LYA']).all()
    # assert (frames['TSNR2_BGS'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_BGS']).all()
    # assert (frames['TSNR2_GPBBACKUP'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBBACKUP']).all()
    # assert (frames['TSNR2_QSO'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_QSO']).all()
    # assert (frames['TSNR2_LRG'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_LRG']).all()
    assert (frames['SURVEY'][frames['EXPID'] == expid] == exposures_fits[k]['SURVEY']).all()
    assert (frames['GOALTYPE'][frames['EXPID'] == expid] == exposures_fits[k]['GOALTYPE']).all()
    assert (frames['FAPRGRM'][frames['EXPID'] == expid] == exposures_fits[k]['FAPRGRM']).all()
    assert (frames['FAFLAVOR'][frames['EXPID'] == expid] == exposures_fits[k]['FAFLAVOR']).all()
    assert (frames['MINTFRAC'][frames['EXPID'] == expid] == exposures_fits[k]['MINTFRAC']).all()
    for column in ('MJD', 'EXPTIME', 'GOALTIME'):
        if column == 'GOALTIME' and (frames[column][frames['EXPID'] == expid] == 0).all():
            print(f"GOALTIME discrepancy for {expid}.")
        else:
            try:
                assert (np.around(frames[column][frames['EXPID'] == expid], 2) == np.around(exposures_fits[k][column], 2)).all()
            except AssertionError:
                try:
                    assert (np.around(frames[column][frames['EXPID'] == expid], 3) == np.around(exposures_fits[k][column], 3)).all()
                except AssertionError:
                    try:
                        assert (np.around(frames[column][frames['EXPID'] == expid], 4) == np.around(exposures_fits[k][column], 4)).all()
                    except AssertionError:
                        pass
                        # print(column, expid, exposures_fits[k][column], frames[column][frames['EXPID'] == expid].tolist())

### Compare tiles to exposures

In [ ]:
for row in tiles_fits:
    w = exposures_fits['TILEID'] == row['TILEID']
    assert len(exposures_fits[w]) == row['NEXP']
    for column in ('SURVEY', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE'):
        try:
            assert (exposures_fits[w][column] == row[column]).all()
        except AssertionError:
            print(row['TILEID'], row[column])
            print(exposures_fits[w][column])
    for column in ('TILERA', 'TILEDEC'):
        try:
            assert (np.around(exposures_fits[w][column], 2) == row[column]).all()
        except AssertionError:
            try:
                assert (np.around(exposures_fits[w][column], 3) == row[column]).all()
            except AssertionError:
                assert (np.around(exposures_fits[w][column], 4) == row[column]).all()
    for column in ('EXPTIME', 'EFFTIME_SPEC', 'LRG_EFFTIME_DARK', 'ELG_EFFTIME_DARK', 'BGS_EFFTIME_BRIGHT', 'LYA_EFFTIME_DARK'):
        try:
            assert np.allclose(np.around(exposures_fits[w][column].sum(), 1), row[column])
        except AssertionError:
            print(row['TILEID'], row[column])
            print(column, np.around(exposures_fits[w][column].sum(), 1))
    for column in ('EFFTIME_ETC', 'EFFTIME_GFA'):
        if (exposures_fits[w][column] == 0).any():
            assert row[column] == 0
        else:
            try:
                assert np.allclose(np.around(exposures_fits[w][column].sum(), 1), row[column])
            except AssertionError:
                print(row['TILEID'], row[column])
                print(column, np.around(exposures_fits[w][column].sum(), 1))
    for column in ('GOALTIME', 'MINTFRAC'):
        try:
            assert np.allclose(np.around(exposures_fits[w][column], 1), row[column])
        except AssertionError:
            try:
                assert np.allclose(np.around(exposures_fits[w][column], 2), row[column])
            except AssertionError:
                print(row['TILEID'], row[column])
                print(column, np.round(exposures_fits[w][column], decimals=1))
    
    assert exposures_fits[w]['NIGHT'].max() == row['LASTNIGHT']